In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, BatchNormalization
from tensorflow.keras.utils import plot_model,to_categorical

In [2]:
data_dir = Path("lemon_dataset/")
images = []
labels = []

label_name = {'empty_background': 0, 'bad_quality': 1, 'good_quality': 2}
for x in os.walk(data_dir):
    if x[0] == 'lemon_dataset':
        continue
    label = x[0].split('\\')[1]
    for file in os.listdir(x[0]):
        imagePath = f"{x[0]}\\{file}"
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (227,227))
        if image.shape[2] == 1:
            img = np.dstack([image,image,image])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.astype(np.float32)/255
        
        images.append(image)
        labels.append(label_name[label])
print("Data has been processed")

Data has been processed


In [3]:
model = keras.models.Sequential()
num_output = 3
model.add(Conv2D(filters = 96, input_shape = (227, 227, 3), 
            kernel_size = (11, 11), strides = (4, 4), 
            padding = 'valid'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size =(2,2), strides = (2,2), padding='valid'))

model.add(BatchNormalization())

model.add(Conv2D(filters = 256, kernel_size = (11, 11), 
            strides = (1, 1), padding = 'valid'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), 
            padding = 'valid'))

model.add(BatchNormalization())

model.add(Conv2D(filters = 384, kernel_size = (3, 3), 
            strides = (1, 1), padding = 'valid'))

model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(Conv2D(filters = 384, kernel_size = (3, 3), 
            strides = (1, 1), padding = 'valid'))

model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(Conv2D(filters = 256, kernel_size = (3, 3), 
            strides = (1, 1), padding = 'valid'))

model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), 
            padding = 'valid'))

model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(4096, input_shape = (224*224*3, )))
model.add(Activation('relu'))

model.add(Dropout(0.4))

model.add(BatchNormalization())

model.add(Dense(4096))
model.add(Activation('relu'))


model.add(Dropout(0.4))

model.add(BatchNormalization())
  

model.add(Dense(num_output))
model.add(Activation('softmax'))
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [4]:
total_history = []
n_folds = 5
for x in range(n_folds):
    (x_train, x_test, y_train, y_test) = train_test_split(images, labels, test_size = 0.3)
    y_train = to_categorical(y_train, num_classes = 3)
    y_test = to_categorical(y_test, num_classes = 3)
    x_train = np.asarray(x_train)
    x_test = np.asarray(x_test)
    print(f"Current Fold is {x+1}")
    total_history.append(model.fit(x=x_train,y= y_train, 
                            epochs = 10, 
                            batch_size=128, validation_data= (x_test,y_test)))

Current Fold is 1
Epoch 1/10
14/14 [==============================] - 24s 2s/step - loss: 2.2471 - accuracy: 0.7564 - val_loss: 281.4256 - val_accuracy: 0.5837
Epoch 2/10
14/14 [==============================] - 23s 2s/step - loss: 0.3458 - accuracy: 0.8932 - val_loss: 135.3538 - val_accuracy: 0.4572
Epoch 3/10
14/14 [==============================] - 23s 2s/step - loss: 0.1465 - accuracy: 0.9412 - val_loss: 49.7028 - val_accuracy: 0.4559
Epoch 4/10
14/14 [==============================] - 23s 2s/step - loss: 0.1393 - accuracy: 0.9457 - val_loss: 10.5484 - val_accuracy: 0.4625
Epoch 5/10
14/14 [==============================] - 23s 2s/step - loss: 0.1353 - accuracy: 0.9480 - val_loss: 4.6440 - val_accuracy: 0.4756
Epoch 6/10
14/14 [==============================] - 23s 2s/step - loss: 0.1189 - accuracy: 0.9536 - val_loss: 4.2754 - val_accuracy: 0.5138
Epoch 7/10
14/14 [==============================] - 23s 2s/step - loss: 0.1268 - accuracy: 0.9525 - val_loss: 3.5022 - val_accuracy: 0.5